# Praxisübung: Pipelines
In dieser Übung sollen Sie für den Titanic Datensatz mittels Pipelines und ColumnTransformers die Datenvorverarbeitung und das Modelltraining inkl. Hyperparameteroptimierung durchführen. Starten Sie mit folgendem Code:

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split

df = pd.read_csv("Daten/titanic.csv")

# Label: Überlebt ja/nein
y = df.pop("Survived")

# Split in Trainings- und Testset
X_train, X_test, y_train, y_test = train_test_split(df, y, test_size=0.2, random_state=42)

## ✏ Aufgabe
Benutzen Sie ColumnTransformers und eine Pipeline, um die folgenden Schritte durchzuführen:
1. One-hot encoding der Spalten [`Sex`, `Embarked`]
2. nan's der Spalte age durch Mittelwert ersetzen
3. Skalieren des Features `Age`
4. Entfernen der Spalten [`PassengerId`, `Name`,`Ticket`, `Cabin`]
5. Trainieren eines Entscheidungsbaums. Führen Sie dafür eine Hyperparametersuche nach der besten Kombination aus Baumtiefe und min_samples_split mit Kreuzvalidierung durch.
6. Schließen Sie in die Hyperparametersuche auch noch zwei Strategien zum nan-Auffüllen mit ein: `mean` und `median` 

Werten Sie anschließend die Accuracy sowohl auf den Trainings- als auch auf den Testdaten aus.

In [2]:
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.compose import ColumnTransformer, make_column_transformer
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.tree import DecisionTreeClassifier

# Pipeline soll folgendes machen:
# 1. One-hot encoding der Spalten ["Sex", "Embarked"]
# 2. nan's der Spalte age durch Mittelwert ersetzen
# 3. Skalieren des Features "Age"
# 4. Entfernen der Spalten ["PassengerId", "Name","Ticket", "Cabin"]
# 5. Hyperparametersuche nach bester Kombination aus Baumtiefe und min_samples_split sowie "mean" und "median" als Strategie beim Ersetzen von nan 

column_trans = ColumnTransformer([("spalten", "drop", ["PassengerId", "Name", "Ticket", "Cabin"]),
                                   ("nans", make_pipeline(SimpleImputer(strategy="mean"), StandardScaler()),["Age"]),
                                   ("onehot", OneHotEncoder(handle_unknown="ignore"), ["Sex", "Embarked"])
                                  ], remainder="passthrough")

pipe = Pipeline([("vv", column_trans), 
                 ("dt", DecisionTreeClassifier())])

param_grid = {"vv__nans__simpleimputer__strategy": ["mean", "median"], 
              "dt__max_depth": range(1,5), 
              "dt__min_samples_split": [2,4,8]}

gs = GridSearchCV(pipe, param_grid=param_grid, cv=3)
gs.fit(X_train,y_train)

print(f"Accuracy Trainingsdaten: {gs.score(X_train, y_train)}")
print(f"Accuracy Testdaten: {gs.score(X_test, y_test)}")

Accuracy Trainingsdaten: 0.8342696629213483
Accuracy Testdaten: 0.7988826815642458
